In [13]:
import json

In [14]:
syms = []
symfile = open('../snp500sym.csv','r')
symlines = symfile.readlines()
for line in symlines:
    syms.append(line.strip())
print(syms[:5])
print(f'no of stocks = {len(syms)}')

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD']
no of stocks = 505


In [34]:
bal = open('INCOME_STATEMENT/AAL.json', 'r')
inc = open('BALANCE_SHEET/AAL.json', 'r')
bal = json.load(bal)
inc = json.load(inc)
inc = inc['annualReports'][0]
bal = bal['annualReports'][0]
income_statement_indicators = [u[0] for u in inc.items()]
balance_sheet_indicators = [u[0] for u in bal.items()]
income_statement_indicators = set(income_statement_indicators)
balance_sheet_indicators = set(balance_sheet_indicators)
# print(income_statement_indicators)
# print(balance_sheet_indicators)
indicators = income_statement_indicators.union(balance_sheet_indicators)
print(indicators)
# print(len(indicators))


{'deferredRevenue', 'shortTermDebt', 'depreciationAndAmortization', 'retainedEarnings', 'inventory', 'goodwill', 'netIncomeFromContinuingOperations', 'cashAndShortTermInvestments', 'longTermDebt', 'otherCurrentLiabilities', 'nonInterestIncome', 'totalNonCurrentAssets', 'accumulatedDepreciationAmortizationPPE', 'otherNonOperatingIncome', 'capitalLeaseObligations', 'investmentIncomeNet', 'ebitda', 'totalLiabilities', 'currentLongTermDebt', 'totalNonCurrentLiabilities', 'intangibleAssetsExcludingGoodwill', 'treasuryStock', 'currentNetReceivables', 'researchAndDevelopment', 'fiscalDateEnding', 'commonStock', 'costOfRevenue', 'incomeTaxExpense', 'currentAccountsPayable', 'interestAndDebtExpense', 'comprehensiveIncomeNetOfTax', 'shortTermInvestments', 'totalCurrentLiabilities', 'otherNonCurrrentAssets', 'operatingExpenses', 'totalAssets', 'operatingIncome', 'cashAndCashEquivalentsAtCarryingValue', 'currentDebt', 'longTermDebtNoncurrent', 'totalShareholderEquity', 'grossProfit', 'ebit', 'tota

In [54]:
period_file = open(f'../data/QuarterlyData/2021_Q4.csv', 'r')
indicators = period_file.readline()
lines = period_file.readlines()
ticker = []
for line in lines:
    line = line.strip().split(',')
    ticker.append(line[0])
indicators = indicators.strip().split(',')
# print(indicators)
indicatos_ind = dict()
for i in range(len(indicators)):
    indicatos_ind[indicators[i].lower()] = i+1
print(indicatos_ind)

{'ticker': 1, 'reportedcurrency': 2, 'totalassets': 3, 'totalcurrentassets': 4, 'cashandcashequivalentsatcarryingvalue': 5, 'cashandshortterminvestments': 6, 'inventory': 7, 'currentnetreceivables': 8, 'totalnoncurrentassets': 9, 'propertyplantequipment': 10, 'accumulateddepreciationamortizationppe': 11, 'intangibleassets': 12, 'intangibleassetsexcludinggoodwill': 13, 'goodwill': 14, 'investments': 15, 'longterminvestments': 16, 'shortterminvestments': 17, 'othercurrentassets': 18, 'othernoncurrrentassets': 19, 'totalliabilities': 20, 'totalcurrentliabilities': 21, 'currentaccountspayable': 22, 'deferredrevenue': 23, 'currentdebt': 24, 'shorttermdebt': 25, 'totalnoncurrentliabilities': 26, 'capitalleaseobligations': 27, 'longtermdebt': 28, 'currentlongtermdebt': 29, 'longtermdebtnoncurrent': 30, 'shortlongtermdebttotal': 31, 'othercurrentliabilities': 32, 'othernoncurrentliabilities': 33, 'totalshareholderequity': 34, 'treasurystock': 35, 'retainedearnings': 36, 'commonstock': 37, 'com

In [72]:
ratios = [
    ['pe', [], []], 
    ['eps', [], []],
    ['de', ['totalLiabilities'], ['totalShareHolderEquity']], 
    ['da', ['totalLiabilities'], ['totalAssets']],
    ['roe', ['netincome'], ['totalShareHolderEquity']],
    ['roa', ['netincome'], ['totalAssets']],
    ['current_ratio', ['totalCurrentAssets'], ['totalCurrentLiabilities']],
    ['quick_ratio', ['cashandshortterminvestments', 'accountsreceivable'], ['totalcurrentliabilities']],
    ['asset_turnover', ['totalrevenue'], ['totalAssets']],
    ['debt_ratio', ['totalliabilities'], ['totalassets']],
]

In [73]:
for year in range(2017, 2022):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyData/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        info = []
        for line in lines:
            line = line.strip()
            line = line.split(',')
            info.append(line)
        # print(info[:5])
        period_ratios = open(f'../data/QuarterlyRatios/{period}.csv', 'w')
        ratio_header = ['ticker']
        for ratio in ratios:
            ratio_header.append(ratio[0])
        period_ratios.write(','.join(ratio_header)  + '\n')
        for i in range(len(ticker)):
            to_write = [ticker[i]]
            for ratio in ratios:
                
                num = 0
                den = 0
                    
                try:
                    for nums in ratio[1]:
                        num += float(info[i][indicatos_ind[nums.lower()]])
                    for dens in ratio[2]:
                        den += float(info[i][indicatos_ind[dens.lower()]])
                    to_write.append(str(num/den))
                    pass
                except:
                    if num != 0 and den != 0:
                        print(num, den)
                    to_write.append('#####')
                    pass
            
            period_ratios.write(','.join(to_write) + '\n')